# 모델 매개변수 최적화
- 데이터에 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트 하기
- 모델을 학습하는 과정은 반복적인 과정을 거친다. (epoch라 부른다.)
- 반복 단계에서 모델은 출력을 추측하고, 추측과 정답 사이의 오류(loss)를 계산한다. 
- 매개변수에 대한 오류의 도함수(derivative)를 수집한 뒤, 경사하강법을 사용하여 이 parameter들을 최적화한다.

In [21]:
"""이전 step에서 했던 코드들"""
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64, drop_last=True)
test_dataloader = DataLoader(test_data, batch_size=64, drop_last=True)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

    
"""이전 step에서 했던 코드들"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork().to(device)

### Hyperparameter
- 하이퍼파라미터는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수이다.
- 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율(converence rate)에 영향을 미칠 수 있다.
- 하이퍼파라미터 정의
    - 에폭(epoch) 수 : 데이터 셋을 반복하는 횟수
    - 배치 크기 (batch size) : 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
    - 학습률 (learning rate) : 각 배치/epoch에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있다.

In [12]:
learning_rate = 0.003
batch_size = 64
epochs = 5

--------

### 최적화 단계
- 하이퍼파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있다. (최적화 단계의 각 반복(iteration)을 `에폭(epoch)`이라고 부른다.)
- 하나의 에폭은 다음 두 부분으로 구성된다.
    1. 학습 단계 : train data set(학습용 데이터셋)을 반복(iterate)하고 최적의 매개변수로 수렴한다.
    2. 검증/테스트 단계 : 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iterate)한다.

#### 손실 함수(loss function)
- 학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높다.
- 손실 함수(loss fuction)는 획득한 결과와 실제 값 사이의 틀린 정도를 측정하며, 학습중에 이 값을 최소화하려고 한다. (label과 pred를 비교하여 loss를 계산)
- 모델의 출렬 logit을 손실 함수에 전달하여 logit을 정규화하고 예측 오류를 계산한다.

In [13]:
loss_fn = nn.CrossEntropyLoss()

#### 옵티마이저 (Optimizer)
- 최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수(weight, bias)를 조정하는 과정이다.
- 최적화 알고리즘은 이 과정이 수행되는 방식을 정의
    - 최적화 알고리즘에는 많은 종류가 있다. (이 경우 SGD 사용)

In [14]:
"""학습하려는 모델의 매개변수와 학습률(learning rate), 하이퍼파라미터를 등록하여 옵티마이저를 초기화한다."""
opimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

#### 학습 단계에서 최적화는 세 단계로 이루어진다.

1. `optimizer.zero_grad()`를 호출하여 `모델 매개변수의 변화도를 재설정`한다.
    - 기본적으로 변화도는 더해지기 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정
    
2. `loss.backward()`를 호출하여 `예측 손실을 역전파`한다. 
    - pytorch는 각 매개변수에 대한 손실의 변화도를 저장한다.

3. 변화도를 계산한 뒤에는 `optimizer.step()`을 호출하여 `역전파 단계에서 수집된 변화도로 매개변수를 조정`한다.

----

#### 전체 구현
- 최적화 코드를 반복하여 수행 : train_loop
- 테스트 데이터로 모델의 성능을 측정 : test_loop

In [25]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        
        ## 예측 (prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y) # 손실값 계산
        
        ## 역전파
        optimizer.zero_grad() # 모델 매개변수의 변화도 재설정
        loss.backward() # 예측 손실 역전파
        optimizer.step() # 역전파 단계에서 수집된 변화도로 매개변수 조정
        
        if batch%100==0:
            loss, current = loss.item(), batch*len(X) # 100번마다 batch별 loss 값 확인
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # 전체 데이터 개수
    num_batches = len(dataloader) # batch 개수
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(dim = 1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches # batch 별 평균 loss
    correct /= size # 전체 test data 중에서 맞은 개수 확인
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn) # inference 시에 optimizer는 불필요
print("Done")

Epoch 1
-------------------------------
loss: 2.310904  [    0/60000]
loss: 2.271514  [ 6400/60000]
loss: 2.216685  [12800/60000]
loss: 2.191461  [19200/60000]
loss: 2.123796  [25600/60000]
loss: 2.024082  [32000/60000]
loss: 1.986719  [38400/60000]
loss: 1.823268  [44800/60000]
loss: 1.757781  [51200/60000]
loss: 1.577937  [57600/60000]
Test Error: 
 Accuracy: 60.1%, Avg loss: 1.548143 

Epoch 2
-------------------------------
loss: 1.617948  [    0/60000]
loss: 1.510161  [ 6400/60000]
loss: 1.290626  [12800/60000]
loss: 1.324711  [19200/60000]
loss: 1.146088  [25600/60000]
loss: 1.139155  [32000/60000]
loss: 1.138788  [38400/60000]
loss: 1.039635  [44800/60000]
loss: 1.056428  [51200/60000]
loss: 0.972953  [57600/60000]
Test Error: 
 Accuracy: 65.6%, Avg loss: 0.976325 

Epoch 3
-------------------------------
loss: 1.033526  [    0/60000]
loss: 1.039214  [ 6400/60000]
loss: 0.820355  [12800/60000]
loss: 0.973472  [19200/60000]
loss: 0.821621  [25600/60000]
loss: 0.847630  [32000/600

In [28]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[ 0.0248,  0.0094,  0.0336,  ...,  0.0028,  0.0300,  0.0082],
                      [ 0.0122,  0.0120, -0.0343,  ..., -0.0235, -0.0050, -0.0122],
                      [ 0.0187, -0.0114,  0.0136,  ...,  0.0224,  0.0055,  0.0029],
                      ...,
                      [ 0.0122,  0.0339, -0.0101,  ...,  0.0075, -0.0175,  0.0329],
                      [ 0.0090, -0.0021,  0.0293,  ...,  0.0200,  0.0103, -0.0305],
                      [-0.0166, -0.0238, -0.0070,  ..., -0.0097,  0.0269, -0.0331]],
                     device='cuda:0')),
             ('linear_relu_stack.0.bias',
              tensor([-0.0218, -0.0413,  0.0179, -0.0145,  0.0029, -0.0590,  0.0236, -0.0268,
                      -0.0047,  0.0570, -0.0003, -0.0379,  0.0467, -0.0470,  0.0036,  0.0139,
                      -0.0286, -0.0039, -0.0054,  0.0204, -0.0744,  0.0155, -0.0141, -0.0008,
                       0.0747, -0.0305,  0.0523,  0.0507, -0